In [31]:
import psycopg2
from psycopg2 import sql

In [32]:
arquivo = "unitech.sql"

In [33]:
def ler_arquivo_sql(nome_arquivo):
    with open(nome_arquivo, 'r') as arquivo:
        sql = arquivo.read()    
    return sql   

In [34]:
def criar_tabelas(cursor, arquivo_sql):
    try:
        query = ler_arquivo_sql(arquivo_sql)
        cursor.execute(query)
        print('Tabelas criadas com sucesso!')
    except Exception as e:
        print('Erro ao criar tabelas: ', e)

In [35]:
def conecta_db(database, user, password, host):
  con = psycopg2.connect(host=host, 
                         database=database,
                         user=user, 
                         password=password)
  return con

In [36]:
# Função para deletar todas as tabelas do banco de dado
def delete_all_tables(cur):
    try:
        
        # Consultar todas as tabelas no esquema público
        cur.execute("""
            SELECT table_name
            FROM information_schema.tables
            WHERE table_schema = 'public'
        """)
        
        # Buscar todos os nomes de tabelas
        tables = cur.fetchall()
        
        # Desabilitar verificação de chave estrangeira temporariamente
        cur.execute("SET session_replication_role = 'replica';")
        
        # Deletar cada tabela
        for table in tables:
            cur.execute(sql.SQL("DROP TABLE IF EXISTS {} CASCADE").format(
                sql.Identifier(table[0])
            ))
        
        # Habilitar verificação de chave estrangeira novamente
        cur.execute("SET session_replication_role = 'origin';")
        
        # Confirmar as alterações no banco de dados
        conn.commit()
        
        # Fechar a comunicação com o banco de dados
        cur.close()
        print("Todas as tabelas foram deletadas com sucesso!")
    
    except Exception as e:
        print(f"Erro: {e}")

In [38]:
# Realizando conexão com o banco de dados

db_config = {
    'database': 'unitech',
    'user': 'postgres',
    'password': 'admin',
    'host': 'localhost'
}

conn = conecta_db(**db_config)

cur = conn.cursor()

In [39]:
# Criando o bando de dados e as tabelas a partir do arquivo sql

criar_tabelas(cur, arquivo)


conn.commit()
cur.close()

Tabelas criadas com sucesso!


In [ ]:
# Em caso de testes pode-se deletar todas as tabelas do banco de dados com os seguintes comandos:

# conn.commit()
# delete_all_tables(cur)